In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Charger les données
df = pd.read_csv("car data.csv")
print("=== DONNÉES ORIGINALES ===")
print(f"Shape initiale: {df.shape}")
print(f"Colonnes: {list(df.columns)}")

# Faire une copie pour le nettoyage
df_clean = df.copy()

=== DONNÉES ORIGINALES ===
Shape initiale: (301, 9)
Colonnes: ['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type', 'Transmission', 'Owner']


In [3]:
def analyze_data_quality(df):
    """Analyse complète de la qualité des données"""
    print("\n=== ANALYSE DE QUALITÉ ===")
    
    # Informations générales
    print(f"Shape: {df.shape}")
    print(f"Mémoire utilisée: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Valeurs manquantes
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Colonne': missing_data.index,
        'Valeurs_manquantes': missing_data.values,
        'Pourcentage': missing_percent.values
    }).sort_values('Pourcentage', ascending=False)
    
    print("\nValeurs manquantes:")
    print(missing_df[missing_df['Valeurs_manquantes'] > 0])
    
    # Doublons
    duplicates = df.duplicated().sum()
    print(f"\nDoublons complets: {duplicates}")
    
    # Types de données
    print(f"\nTypes de données:")
    for col in df.columns:
        print(f"  {col}: {df[col].dtype} - Valeurs uniques: {df[col].nunique()}")
    
    return missing_df

# Analyse initiale
quality_report = analyze_data_quality(df_clean)



=== ANALYSE DE QUALITÉ ===
Shape: (301, 9)
Mémoire utilisée: 0.09 MB

Valeurs manquantes:
Empty DataFrame
Columns: [Colonne, Valeurs_manquantes, Pourcentage]
Index: []

Doublons complets: 2

Types de données:
  Car_Name: object - Valeurs uniques: 98
  Year: int64 - Valeurs uniques: 16
  Selling_Price: float64 - Valeurs uniques: 156
  Present_Price: float64 - Valeurs uniques: 148
  Driven_kms: int64 - Valeurs uniques: 206
  Fuel_Type: object - Valeurs uniques: 3
  Selling_type: object - Valeurs uniques: 2
  Transmission: object - Valeurs uniques: 2
  Owner: int64 - Valeurs uniques: 3


In [4]:
def clean_numerical_outliers(df, column, method='iqr', factor=1.5):
    """Nettoyer les outliers numériques"""
    if method == 'iqr':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        
        outliers_mask = (df[column] < lower_bound) | (df[column] > upper_bound)
        
    elif method == 'zscore':
        z_scores = np.abs(stats.zscore(df[column].dropna()))
        outliers_mask = z_scores > factor
    
    outliers_count = outliers_mask.sum()
    print(f"  {column}: {outliers_count} outliers détectés ({outliers_count/len(df)*100:.2f}%)")
    
    return outliers_mask

# Identifier les colonnes numériques
numeric_columns = df_clean.select_dtypes(include=[np.number]).columns.tolist()
target_col = 'Selling_Price'

print("\n=== DÉTECTION DES OUTLIERS ===")
outliers_masks = {}

for col in numeric_columns:
    if col in df_clean.columns:
        # Exclure les valeurs négatives ou nulles pour les prix et années
        if col in ['Selling_Price', 'Present_Price'] and (df_clean[col] <= 0).any():
            print(f"⚠️  {col}: Valeurs négatives/nulles détectées")
            df_clean = df_clean[df_clean[col] > 0]
        
        if col == 'Year' and (df_clean[col] < 1900).any():
            print(f"⚠️  {col}: Années suspectes détectées")
            df_clean = df_clean[df_clean[col] >= 1900]
        
        outliers_masks[col] = clean_numerical_outliers(df_clean, col)


=== DÉTECTION DES OUTLIERS ===
  Year: 7 outliers détectés (2.33%)
  Selling_Price: 17 outliers détectés (5.65%)
  Present_Price: 14 outliers détectés (4.65%)
  Driven_kms: 8 outliers détectés (2.66%)
  Owner: 11 outliers détectés (3.65%)


In [5]:
def remove_extreme_outliers(df, outliers_masks, threshold=0.05):
    """
    Supprime les outliers extrêmes pour chaque colonne numé­rique,
    en étant plus tolérant si la proportion dépasse un certain seuil.

    Paramètres :
    - df : DataFrame d'origine
    - outliers_masks : dictionnaire {colonne: masque booléen des outliers}
    - threshold : seuil de tolérance (par défaut 5%) pour décider d'être plus conservateur

    Retour :
    - df nettoyé des outliers extrêmes
    """
    df_no_outliers = df.copy()
    
    for col, mask in outliers_masks.items():
        proportion_outliers = mask.sum() / len(df)
        if proportion_outliers > threshold:
            # Conserver uniquement les 5% des outliers les plus extrêmes
            outlier_values = df[mask][col]
            extreme_threshold = np.percentile(outlier_values, 95)
            extreme_mask = mask & (df[col] > extreme_threshold)
            df_no_outliers = df_no_outliers[~extreme_mask]
            print(f"  {col}: {extreme_mask.sum()} outliers extrêmes supprimés (>{threshold*100:.0f}%)")
        else:
            df_no_outliers = df_no_outliers[~mask]
            print(f"  {col}: {mask.sum()} outliers supprimés (<={threshold*100:.0f}%)")
    
    return df_no_outliers


# ===== Exécution du nettoyage des outliers =====
print("\n=== TRAITEMENT DES OUTLIERS ===")
df_clean = remove_extreme_outliers(df_clean, outliers_masks)

print(f"\nShape après nettoyage des outliers : {df_clean.shape}")



=== TRAITEMENT DES OUTLIERS ===
  Year: 7 outliers supprimés (<=5%)
  Selling_Price: 1 outliers extrêmes supprimés (>5%)
  Present_Price: 14 outliers supprimés (<=5%)
  Driven_kms: 8 outliers supprimés (<=5%)
  Owner: 11 outliers supprimés (<=5%)

Shape après nettoyage des outliers : (268, 9)


In [6]:
def clean_categorical_data(df):
    """Nettoyer les données catégorielles"""
    df_cat_clean = df.copy()
    
    print("\n=== NETTOYAGE CATÉGORIEL ===")
    
    categorical_columns = df.select_dtypes(include=['object']).columns
    
    for col in categorical_columns:
        if col in df_cat_clean.columns:
            print(f"\n{col}:")
            print(f"  Valeurs uniques avant: {df_cat_clean[col].nunique()}")
            
            # Nettoyer les espaces et la casse
            df_cat_clean[col] = df_cat_clean[col].astype(str).str.strip().str.lower()
            
            # Remplacer les valeurs communes
            df_cat_clean[col] = df_cat_clean[col].replace({
                'nan': np.nan,
                'unknown': np.nan,
                'other': np.nan,
                '': np.nan
            })
            
            # Regrouper les catégories rares (moins de 1% des données)
            value_counts = df_cat_clean[col].value_counts()
            rare_categories = value_counts[value_counts < len(df_cat_clean) * 0.01].index
            
            if len(rare_categories) > 0:
                df_cat_clean[col] = df_cat_clean[col].replace(rare_categories, 'other_rare')
                print(f"  {len(rare_categories)} catégories rares regroupées")
            
            print(f"  Valeurs uniques après: {df_cat_clean[col].nunique()}")
            print(f"  Top 5 valeurs: {df_cat_clean[col].value_counts().head().to_dict()}")
    
    return df_cat_clean

df_clean = clean_categorical_data(df_clean)


=== NETTOYAGE CATÉGORIEL ===

Car_Name:
  Valeurs uniques avant: 89
  58 catégories rares regroupées
  Valeurs uniques après: 32
  Top 5 valeurs: {'other_rare': 75, 'city': 26, 'corolla altis': 16, 'verna': 14, 'brio': 10}

Fuel_Type:
  Valeurs uniques avant: 3
  1 catégories rares regroupées
  Valeurs uniques après: 3
  Top 5 valeurs: {'petrol': 220, 'diesel': 46, 'other_rare': 2}

Selling_type:
  Valeurs uniques avant: 2
  Valeurs uniques après: 2
  Top 5 valeurs: {'dealer': 175, 'individual': 93}

Transmission:
  Valeurs uniques avant: 2
  Valeurs uniques après: 2
  Top 5 valeurs: {'manual': 243, 'automatic': 25}


In [7]:
def create_engineered_features(df):
    """Créer des features engineered pour réduire l'overfitting"""
    df_eng = df.copy()
    
    print("\n=== FEATURE ENGINEERING ===")
    
    # Age de la voiture (plus informatif que l'année seule)
    if 'Year' in df_eng.columns:
        current_year = 2024  # Ajustez selon vos données
        df_eng['Car_Age'] = current_year - df_eng['Year']
        print("✅ Car_Age créé")
    
    # Ratio de dépréciation
    if 'Present_Price' in df_eng.columns and 'Selling_Price' in df_eng.columns:
        df_eng['Depreciation_Ratio'] = (df_eng['Present_Price'] - df_eng['Selling_Price']) / df_eng['Present_Price']
        df_eng['Depreciation_Ratio'] = df_eng['Depreciation_Ratio'].clip(0, 1)  # Limiter entre 0 et 1
        print("✅ Depreciation_Ratio créé")
    
    # Binning de l'âge en catégories
    if 'Car_Age' in df_eng.columns:
        df_eng['Age_Category'] = pd.cut(df_eng['Car_Age'], 
                                       bins=[0, 2, 5, 10, float('inf')], 
                                       labels=['Nouveau', 'Recent', 'Moyen', 'Ancien'])
        print("✅ Age_Category créé")
    
    # Log transformation pour les prix (réduire la variance)
    price_columns = ['Selling_Price', 'Present_Price']
    for col in price_columns:
        if col in df_eng.columns:
            df_eng[f'{col}_log'] = np.log1p(df_eng[col])
            print(f"✅ {col}_log créé")
    
    return df_eng

df_clean = create_engineered_features(df_clean)


=== FEATURE ENGINEERING ===
✅ Car_Age créé
✅ Depreciation_Ratio créé
✅ Age_Category créé
✅ Selling_Price_log créé
✅ Present_Price_log créé


In [ ]:
5. DÉTECTION ET SUPPRESSION DES DOUBLONS SOPHISTIQUÉS

In [8]:
def remove_sophisticated_duplicates(df):
    """Supprimer les doublons sophistiqués"""
    df_dedup = df.copy()
    
    print("\n=== SUPPRESSION DOUBLONS SOPHISTIQUÉS ===")
    
    # Doublons exacts
    exact_dups = df_dedup.duplicated().sum()
    df_dedup = df_dedup.drop_duplicates()
    print(f"Doublons exacts supprimés: {exact_dups}")
    
    # Doublons basés sur des colonnes clés (exemple)
    key_columns = ['Year', 'Present_Price', 'Fuel_Type', 'Transmission']
    if all(col in df_dedup.columns for col in key_columns):
        before_count = len(df_dedup)
        df_dedup = df_dedup.drop_duplicates(subset=key_columns)
        key_dups = before_count - len(df_dedup)
        print(f"Doublons par colonnes clés supprimés: {key_dups}")
    
    return df_dedup

df_clean = remove_sophisticated_duplicates(df_clean)


=== SUPPRESSION DOUBLONS SOPHISTIQUÉS ===
Doublons exacts supprimés: 1
Doublons par colonnes clés supprimés: 44


In [9]:
def validate_data_consistency(df):
    """Valider la cohérence des données"""
    df_valid = df.copy()
    
    print("\n=== VALIDATION DE COHÉRENCE ===")
    
    initial_count = len(df_valid)
    
    # Selling_Price ne peut pas être supérieur à Present_Price (dans la plupart des cas)
    if 'Selling_Price' in df_valid.columns and 'Present_Price' in df_valid.columns:
        inconsistent = df_valid['Selling_Price'] > df_valid['Present_Price'] * 1.2  # 20% de marge
        df_valid = df_valid[~inconsistent]
        print(f"Lignes avec prix de vente > 120% prix présent supprimées: {inconsistent.sum()}")
    
    # Années futures
    if 'Year' in df_valid.columns:
        future_years = df_valid['Year'] > 2024
        df_valid = df_valid[~future_years]
        print(f"Voitures avec année future supprimées: {future_years.sum()}")
    
    # Kilométrage négatif (si cette colonne existe)
    km_columns = [col for col in df_valid.columns if 'km' in col.lower() or 'mileage' in col.lower()]
    for col in km_columns:
        negative_km = df_valid[col] < 0
        df_valid = df_valid[~negative_km]
        print(f"Lignes avec {col} négatif supprimées: {negative_km.sum()}")
    
    final_count = len(df_valid)
    print(f"Total lignes supprimées pour incohérence: {initial_count - final_count}")
    
    return df_valid

df_clean = validate_data_consistency(df_clean)


=== VALIDATION DE COHÉRENCE ===
Lignes avec prix de vente > 120% prix présent supprimées: 0
Voitures avec année future supprimées: 0
Lignes avec Driven_kms négatif supprimées: 0
Total lignes supprimées pour incohérence: 0


In [10]:
def final_cleanup_and_report(df_original, df_clean):
    """Nettoyage final et rapport de synthèse"""
    
    print("\n" + "="*60)
    print("RAPPORT DE NETTOYAGE FINAL")
    print("="*60)
    
    print(f"Shape originale: {df_original.shape}")
    print(f"Shape nettoyée: {df_clean.shape}")
    print(f"Lignes supprimées: {len(df_original) - len(df_clean)} ({(len(df_original) - len(df_clean))/len(df_original)*100:.1f}%)")
    
    # Valeurs manquantes restantes
    missing_final = df_clean.isnull().sum()
    if missing_final.sum() > 0:
        print(f"\nValeurs manquantes restantes:")
        for col, count in missing_final[missing_final > 0].items():
            print(f"  {col}: {count} ({count/len(df_clean)*100:.1f}%)")
    else:
        print("\n✅ Aucune valeur manquante restante!")
    
    # Distribution de la variable cible
    if 'Selling_Price' in df_clean.columns:
        target_stats = df_clean['Selling_Price'].describe()
        print(f"\nStatistiques Selling_Price:")
        print(f"  Moyenne: {target_stats['mean']:.2f}")
        print(f"  Médiane: {target_stats['50%']:.2f}")
        print(f"  Écart-type: {target_stats['std']:.2f}")
        print(f"  Min-Max: {target_stats['min']:.2f} - {target_stats['max']:.2f}")
    
    # Features finales
    print(f"\nFeatures finales ({len(df_clean.columns)}):")
    for col in sorted(df_clean.columns):
        dtype = df_clean[col].dtype
        nunique = df_clean[col].nunique()
        print(f"  {col}: {dtype} ({nunique} valeurs uniques)")
    
    return df_clean

df_final = final_cleanup_and_report(df, df_clean)


RAPPORT DE NETTOYAGE FINAL
Shape originale: (301, 9)
Shape nettoyée: (223, 14)
Lignes supprimées: 78 (25.9%)

✅ Aucune valeur manquante restante!

Statistiques Selling_Price:
  Moyenne: 3.90
  Médiane: 3.35
  Écart-type: 3.42
  Min-Max: 0.10 - 18.00

Features finales (14):
  Age_Category: category (2 valeurs uniques)
  Car_Age: int64 (13 valeurs uniques)
  Car_Name: object (32 valeurs uniques)
  Depreciation_Ratio: float64 (213 valeurs uniques)
  Driven_kms: int64 (166 valeurs uniques)
  Fuel_Type: object (3 valeurs uniques)
  Owner: int64 (1 valeurs uniques)
  Present_Price: float64 (135 valeurs uniques)
  Present_Price_log: float64 (135 valeurs uniques)
  Selling_Price: float64 (121 valeurs uniques)
  Selling_Price_log: float64 (121 valeurs uniques)
  Selling_type: object (2 valeurs uniques)
  Transmission: object (2 valeurs uniques)
  Year: int64 (13 valeurs uniques)


In [11]:
# Sauvegarder les données nettoyées
df_final.to_csv('car_data_cleaned.csv', index=False)
print(f"\n✅ Données nettoyées sauvegardées dans 'car_data_cleaned.csv'")

# Créer un rapport de nettoyage
cleaning_report = {
    'original_shape': df.shape,
    'final_shape': df_final.shape,
    'rows_removed': len(df) - len(df_final),
    'percentage_removed': (len(df) - len(df_final)) / len(df) * 100,
    'columns_added': len(df_final.columns) - len(df.columns)
}

print(f"\n=== UTILISEZ MAINTENANT CES DONNÉES POUR L'ENTRAÎNEMENT ===")
print("# Pour charger les données nettoyées:")
print("df_clean = pd.read_csv('car_data_cleaned.csv')")
print("# Procédez avec votre pipeline de ML habituel")



✅ Données nettoyées sauvegardées dans 'car_data_cleaned.csv'

=== UTILISEZ MAINTENANT CES DONNÉES POUR L'ENTRAÎNEMENT ===
# Pour charger les données nettoyées:
df_clean = pd.read_csv('car_data_cleaned.csv')
# Procédez avec votre pipeline de ML habituel


In [12]:
def create_cleaning_visualizations(df_original, df_clean):
    """Créer des visualisations pour montrer l'impact du nettoyage"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Distribution du prix avant/après
    if 'Selling_Price' in df_original.columns:
        axes[0,0].hist(df_original['Selling_Price'], bins=50, alpha=0.7, label='Original', color='red')
        axes[0,0].hist(df_clean['Selling_Price'], bins=50, alpha=0.7, label='Nettoyé', color='blue')
        axes[0,0].set_title('Distribution Selling_Price')
        axes[0,0].legend()
        axes[0,0].set_xlabel('Prix')
        axes[0,0].set_ylabel('Fréquence')
    
    # Boxplot comparatif
    if 'Selling_Price' in df_original.columns:
        data_comparison = [df_original['Selling_Price'].dropna(), df_clean['Selling_Price'].dropna()]
        axes[0,1].boxplot(data_comparison, labels=['Original', 'Nettoyé'])
        axes[0,1].set_title('Boxplot Selling_Price')
        axes[0,1].set_ylabel('Prix')
    
    # Corrélations avant/après (si suffisamment de colonnes numériques)
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 2:
        corr_clean = df_clean[numeric_cols].corr()
        im = axes[1,0].imshow(corr_clean, cmap='coolwarm', aspect='auto')
        axes[1,0].set_title('Matrice de corrélation (données nettoyées)')
        plt.colorbar(im, ax=axes[1,0])
    
    # Valeurs manquantes avant/après
    missing_orig = df_original.isnull().sum()
    missing_clean = df_clean.isnull().sum()
    
    x_pos = range(len(missing_orig))
    axes[1,1].bar([x-0.2 for x in x_pos], missing_orig.values, width=0.4, label='Original', alpha=0.7)
    axes[1,1].bar([x+0.2 for x in x_pos], missing_clean.values, width=0.4, label='Nettoyé', alpha=0.7)
    axes[1,1].set_title('Valeurs manquantes par colonne')
    axes[1,1].set_xticks(x_pos)
    axes[1,1].set_xticklabels(missing_orig.index, rotation=45)
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.savefig('data_cleaning_report.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("📊 Graphiques de nettoyage sauvegardés dans 'data_cleaning_report.png'")

# Créer les visualisations (optionnel)
# create_cleaning_visualizations(df, df_final)

print("\n🎉 NETTOYAGE TERMINÉ ! Vos données sont maintenant prêtes pour un entraînement sans overfitting.")


🎉 NETTOYAGE TERMINÉ ! Vos données sont maintenant prêtes pour un entraînement sans overfitting.
